In [1]:
import pandas as pd  
import time 
import torch 

In [3]:
df = pd.read_csv('data/simpsons_script_lines.csv')

C:\Users\daimond025\AppData\Local\Temp\ipykernel_281880\3071692218.py:1: DtypeWarning: Columns (4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/simpsons_script_lines.csv')


### Повторим действия преобразования текста

In [4]:
df = pd.read_csv('data/simpsons_script_lines.csv')

C:\Users\daimond025\AppData\Local\Temp\ipykernel_281880\3071692218.py:1: DtypeWarning: Columns (4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/simpsons_script_lines.csv')


In [6]:
phrases = df['normalized_text'].tolist()

In [7]:
text = [[c for c in ph] for ph in phrases if type(ph) is str]

In [11]:
text[0]

['n',
 'o',
 ' ',
 'a',
 'c',
 't',
 'u',
 'a',
 'l',
 'l',
 'y',
 ' ',
 'i',
 't',
 ' ',
 'w',
 'a',
 's',
 ' ',
 'a',
 ' ',
 'l',
 'i',
 't',
 't',
 'l',
 'e',
 ' ',
 'o',
 'f',
 ' ',
 'b',
 'o',
 't',
 'h',
 ' ',
 's',
 'o',
 'm',
 'e',
 't',
 'i',
 'm',
 'e',
 's',
 ' ',
 'w',
 'h',
 'e',
 'n',
 ' ',
 'a',
 ' ',
 'd',
 'i',
 's',
 'e',
 'a',
 's',
 'e',
 ' ',
 'i',
 's',
 ' ',
 'i',
 'n',
 ' ',
 'a',
 'l',
 'l',
 ' ',
 't',
 'h',
 'e',
 ' ',
 'm',
 'a',
 'g',
 'a',
 'z',
 'i',
 'n',
 'e',
 's',
 ' ',
 'a',
 'n',
 'd',
 ' ',
 'a',
 'l',
 'l',
 ' ',
 't',
 'h',
 'e',
 ' ',
 'n',
 'e',
 'w',
 's',
 ' ',
 's',
 'h',
 'o',
 'w',
 's',
 ' ',
 'i',
 't',
 's',
 ' ',
 'o',
 'n',
 'l',
 'y',
 ' ',
 'n',
 'a',
 't',
 'u',
 'r',
 'a',
 'l',
 ' ',
 't',
 'h',
 'a',
 't',
 ' ',
 'y',
 'o',
 'u',
 ' ',
 't',
 'h',
 'i',
 'n',
 'k',
 ' ',
 'y',
 'o',
 'u',
 ' ',
 'h',
 'a',
 'v',
 'e',
 ' ',
 'i',
 't']

In [12]:
CHARS = set('abcdefghijklmnopqrstuvwxyz ') 
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)} 

In [13]:
MAX_LEN = 50 
X = torch.zeros((len(text), MAX_LEN), dtype=int)  
for i in range(len(text)):  
    for j, w in enumerate(text[i]):  
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [15]:
X[0]

tensor([12,  7, 10, 25,  8,  3, 16, 25, 14, 14, 19, 10, 26,  3, 10,  5, 25,  6,
        10, 25, 10, 14, 26,  3,  3, 14, 17, 10,  7,  1, 10, 18,  7,  3, 21, 10,
         6,  7,  4, 17,  3, 26,  4, 17,  6, 10,  5, 21, 17, 12])

Базовая модель  c применением слоя RNN

In [39]:
class Network_base(nn.Module):
    def __init__(self):
        super(Network_base, self).__init__()
        self.embedding = torch.nn.Embedding(28, 30)
        self.rnn = torch.nn.RNN(30, 128)
        self.out = torch.nn.Linear(128, 28)

    def forward(self, sentences, state=None):
        x = self.embedding(sentences)
        x, s = self.rnn(x) 
        return self.out(x)

Построим RNN-ячейку на основе полносвязных слоев

In [22]:
from torch import nn
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size, hidden_size, bias=False)
        self.h2h = nn.Linear(hidden_size, hidden_size)
        self.h2o = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):
        x = self.i2h(input)

        hidden_state = self.h2h(hidden)
        hidden_out = hidden_state.unsqueeze(0)

        hidden_state = torch.tanh(x + hidden_out)
        output = self.h2o(hidden_state)
        return output, hidden_state
        
    @staticmethod
    def initHidden(hidden_size):
        return torch.zeros(1, hidden_size)
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.embedding = torch.nn.Embedding(28, 30)
        self.rnn = RNN(30,  128,  128 )
        self.out = torch.nn.Linear(128, 28)

    def forward(self, sentences, hidden=torch.Tensor):
        x = self.embedding(sentences)
        x, s = self.rnn(x, hidden= hidden)
        return self.out(x)

Обучение

In [23]:
model = Network()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.05)

In [26]:
step_count = 100
for ep in range(10):
    start = time.time()

    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / step_count)):
        optimizer.zero_grad()

        # для каждого батча сформируем скрытй слов слой 
        hidden = RNN.initHidden(hidden_size=128)

        batch = X[i * step_count: (i + 1) * step_count]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()

        answers = model.forward(X_batch, hidden=hidden)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)

        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))

Epoch 0. Time: 4.497, Train loss: 1.686
Epoch 1. Time: 4.457, Train loss: 1.686
Epoch 2. Time: 4.422, Train loss: 1.686
Epoch 3. Time: 4.436, Train loss: 1.686
Epoch 4. Time: 4.188, Train loss: 1.686
Epoch 5. Time: 4.128, Train loss: 1.686
Epoch 6. Time: 4.063, Train loss: 1.685
Epoch 7. Time: 4.047, Train loss: 1.685
Epoch 8. Time: 4.105, Train loss: 1.685
Epoch 9. Time: 4.085, Train loss: 1.685


In [33]:
def generate_sentence(word):
    sentence = list(word)
    sentence = [CHAR_TO_INDEX.get(s, 0) for s in sentence]
    hidden = RNN.initHidden(hidden_size=128)
    
    answers = model.forward(torch.tensor(sentence), hidden=hidden)
    probas, indices = answers.topk(1)
    return ''.join([INDEX_TO_CHAR[ind.item()] for ind in indices.flatten()])

In [34]:
generate_sentence('dog')

' u '

In [35]:
generate_sentence('simpsons')

' nee u  '

### Обучим стандратную ячейку с rnn

In [40]:
model = Network_base()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.05)

In [41]:
for ep in range(10):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 100)):
        # берём батч в 100 элементов
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()

        optimizer.zero_grad()
        answers = model.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))

Epoch 0. Time: 14.067, Train loss: 1.828
Epoch 1. Time: 13.458, Train loss: 1.724
Epoch 2. Time: 12.773, Train loss: 1.707
Epoch 3. Time: 12.671, Train loss: 1.698
Epoch 4. Time: 12.729, Train loss: 1.691
Epoch 5. Time: 12.736, Train loss: 1.685
Epoch 6. Time: 12.624, Train loss: 1.681
Epoch 7. Time: 12.684, Train loss: 1.678
Epoch 8. Time: 12.680, Train loss: 1.675
Epoch 9. Time: 12.624, Train loss: 1.672


In [42]:
def generate_sentence_base(word):
    sentence = list(word)
    sentence = [CHAR_TO_INDEX.get(s, 0) for s in sentence]
    answers = model.forward(torch.tensor(sentence))
    probas, indices = answers.topk(1)
    return ''.join([INDEX_TO_CHAR[ind.item()] for ind in indices.flatten()])

In [43]:
generate_sentence_base('dog')

' uo'

In [45]:
generate_sentence_base('It is')

'none tn '

#### Вывод 
Построили две моделт с rnn ячейкой и  RNN ячейкой на основе полносвязных слоев
Вывод обеих моделей примерно равны 


## Обучим нейронную сеть решать шифр Цезар

In [46]:
import math
from random import shuffle
import time
import pandas as pd
import torch

BATCH_SIZE = 10
STRING_SIZE = 50
NUM_EPOCHS = 20
LEARNING_RATE = 0.05
CAESAR_OFFSET = 2

class Alphabet():

    def __init__(self):
        self.letters = ""

    def __len__(self):
        return len(self.letters)

    def __contains__(self, item):
        return item in self.letters

    def __getitem__(self, item):
        if isinstance(item, int):
            return self.letters[item % len(self.letters)]
        elif isinstance(item, str):
            return self.letters.find(item)

    def __str__(self):
        letters = " ".join(self.letters)
        return f"Alphabet is:\n {letters}\n {len(self)} chars"


    def load_from_df(self,):
        df = pd.read_csv('data/simpsons_script_lines.csv')
        df.dropna(subset=['normalized_text'], inplace=True)
        phrases = df['normalized_text'].tolist()
        for text in phrases:
            for ch in text:
                if type(ch) is str and ch not in self.letters:
                    self.letters += ch
        return self
ALPHABET = Alphabet().load_from_df()

C:\Users\daimond025\AppData\Local\Temp\ipykernel_281880\2458915655.py:36: DtypeWarning: Columns (4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/simpsons_script_lines.csv')


In [50]:
print(ALPHABET)
len(ALPHABET)

Alphabet is:
 n o   a c t u l y i w s e f b h m d g z r k v p j - x 5 q 1 0 9 7 3 ñ 6 8 2 4 _ Ñ é ä ï á à ü è ì ö â D . H " ë ó ! í ç É ' B T õ M ô å ê ú È ? Ù û ł ż ń Ä ǒ ǐ ǎ ē ā ě ī ò E N : ) U ù Ĭ Ö æ ø Ĉ ŭ Ĝ ã W I A R C S F P L
 109 chars


109

In [51]:
class SentenceDataset(torch.utils.data.Dataset):
    def __init__(self, raw_data, alphabet):
        super().__init__()
        self._len = len(raw_data)


        self.y = torch.zeros((len(raw_data), STRING_SIZE), dtype=int)
        for i in range(len(raw_data)):
            for j, ch in enumerate(raw_data[i]):
                if j >= STRING_SIZE:
                    break
                self.y[i, j] = alphabet[ch]

        self.x = torch.tensor([ [i + CAESAR_OFFSET for i in line ] for line in self.y] )

    def __len__(self):
        return self._len

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [56]:
def get_text_array():
    text_array = []
    df = pd.read_csv('data/simpsons_script_lines.csv')
    df.dropna(subset=['normalized_text'], inplace=True)
    phrases = df['normalized_text'].tolist()
    for text in phrases:
        text_array.append(text)
    del text_array[-1]

    return text_array

In [59]:
raw_data = get_text_array()
shuffle(raw_data)
_10_percent = math.ceil(len(raw_data) * 0.1)

val_data = raw_data[:_10_percent]
raw_data = raw_data[_10_percent:]

_20_percent = math.ceil(len(raw_data) * 0.2)
test_data = raw_data[:_20_percent]
train_data = raw_data[_20_percent:]

Y_val = torch.zeros((len(val_data), STRING_SIZE), dtype=int)
for i in range(len(val_data)):
    for j, ch in enumerate(val_data[i]):
        if j >= STRING_SIZE:
            break
        Y_val[i, j] = ALPHABET[ch]

X_val = torch.tensor([[i + CAESAR_OFFSET for i in line] for line in Y_val])

C:\Users\daimond025\AppData\Local\Temp\ipykernel_281880\1714548643.py:3: DtypeWarning: Columns (4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/simpsons_script_lines.csv')


In [60]:
class RNNModel(torch.nn.Module):

    def __init__(self):
        super().__init__()
        self.embed = torch.nn.Embedding(len(ALPHABET) + CAESAR_OFFSET, 32)
        self.rnn = torch.nn.RNN(32, 128, batch_first=True)
        self.linear = torch.nn.Linear(128, len(ALPHABET) + CAESAR_OFFSET)

    def forward(self, sentence, state=None):
        embed = self.embed(sentence)
        o, h = self.rnn(embed)
        return self.linear(o)

In [61]:
train_dl = torch.utils.data.DataLoader(
    SentenceDataset(
        train_data, ALPHABET
    ),
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=True
)
test_dl = torch.utils.data.DataLoader(
    SentenceDataset(
        test_data, ALPHABET
    ),
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=True
)

In [62]:
model = RNNModel()
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)

for epoch in range(NUM_EPOCHS):
    train_loss, train_acc, iter_num = .0, .0, .0
    start_epoch_time = time.time()
    model.train()
    for x_in, y_in in train_dl:
        x_in = x_in
        y_in = y_in.view(1, -1).squeeze()
        optimizer.zero_grad()
        out = model.forward(x_in).view(-1, len(ALPHABET) + CAESAR_OFFSET)
        l = loss(out, y_in)
        train_loss += l.item()
        batch_acc = (out.argmax(dim=1) == y_in)
        train_acc += batch_acc.sum().item() / batch_acc.shape[0]
        l.backward()
        optimizer.step()
        iter_num += 1
    print(
        f"Epoch: {epoch}, loss: {train_loss:.4f}, acc: "
        f"{train_acc / iter_num:.4f}",
        end=" | "
    )
    test_loss, test_acc, iter_num = .0, .0, .0
    model.eval()
    for x_in, y_in in test_dl:
        x_in = x_in
        y_in = y_in.view(1, -1).squeeze()
        out = model.forward(x_in).view(-1, len(ALPHABET) + CAESAR_OFFSET)
        l = loss(out, y_in)
        test_loss += l.item()
        batch_acc = (out.argmax(dim=1) == y_in)
        test_acc += batch_acc.sum().item() / batch_acc.shape[0]
        iter_num += 1
    print(
        f"test loss: {test_loss:.4f}, test acc: {test_acc / iter_num:.4f} | "
        f"{time.time() - start_epoch_time:.2f} sec."
    )


Epoch: 0, loss: 413.4166, acc: 0.9937 | test loss: 9.0692, test acc: 0.9996 | 42.08 sec.
Epoch: 1, loss: 28.9587, acc: 0.9996 | test loss: 5.8772, test acc: 0.9996 | 42.21 sec.
Epoch: 2, loss: 21.1537, acc: 0.9997 | test loss: 4.5531, test acc: 0.9997 | 41.86 sec.
Epoch: 3, loss: 16.9223, acc: 0.9997 | test loss: 3.6964, test acc: 0.9997 | 39.97 sec.
Epoch: 4, loss: 13.9449, acc: 0.9998 | test loss: 3.0580, test acc: 0.9998 | 40.93 sec.
Epoch: 5, loss: 11.7539, acc: 0.9998 | test loss: 2.5800, test acc: 0.9998 | 38.96 sec.
Epoch: 6, loss: 10.1039, acc: 0.9998 | test loss: 2.2140, test acc: 0.9999 | 38.63 sec.
Epoch: 7, loss: 8.8224, acc: 0.9999 | test loss: 1.9263, test acc: 0.9999 | 38.64 sec.
Epoch: 8, loss: 7.8006, acc: 0.9999 | test loss: 1.6952, test acc: 0.9999 | 38.54 sec.
Epoch: 9, loss: 6.9724, acc: 0.9999 | test loss: 1.5082, test acc: 0.9999 | 38.60 sec.
Epoch: 10, loss: 6.2939, acc: 0.9999 | test loss: 1.3552, test acc: 0.9999 | 38.58 sec.
Epoch: 11, loss: 5.7351, acc: 0.99

In [63]:
idx = 10
val_results = model(X_val).argmax(dim=2)
val_acc = (val_results == Y_val).flatten()
val_acc = (val_acc.sum() / val_acc.shape[0]).item()
out_sentence = "".join([ALPHABET[i.item()]  for i in val_results[idx]])
true_sentence = "".join([ALPHABET[i.item()] for i in Y_val[idx]])
print(f"Validation accuracy is : {val_acc:.4f}")
print("-" * 20)
print(f"Validation sentence is: \"{out_sentence}\"")
print("-" * 20)
print(f"True sentence is:       \"{true_sentence}\"")


Validation accuracy is : 1.0000
--------------------
Validation sentence is: "i wanna play dodge-rocknnnnnnnnnnnnnnnnnnnnnnnnnnn"
--------------------
True sentence is:       "i wanna play dodge-rocknnnnnnnnnnnnnnnnnnnnnnnnnnn"


In [65]:
sentence = "simpsons simpsons simpsons simpsons simpsons "
sentence_idx = [ALPHABET[i] for i in sentence]
encrypted_sentence_idx = [i + CAESAR_OFFSET for i in sentence_idx]
encrypted_sentence = "".join([ALPHABET[i] for i in encrypted_sentence_idx])
result = model(torch.tensor([encrypted_sentence_idx])).argmax(dim=2)
deencrypted_sentence = "".join([ALPHABET[i.item()] for i in result.flatten()])
print(f"Encrypted sentence is : {encrypted_sentence}")
print("-" * 20)
print(deencrypted_sentence)

Encrypted sentence is : fsg-fa fcfsg-fa fcfsg-fa fcfsg-fa fcfsg-fa fc
--------------------
simpsons simpsons simpsons simpsons simpsons 
